### 2.1 DataFrame
#### 1. Creating DataFrames
1. `Dataset`:  
 `Datasets`是一种分布式数据集, 拥有RDD的优势(强类型,支持lambda表达式).一个 Dataset 可以从JVM对象来构造并且使用transformation算子
2. `DataFrame`  
  `DataFrame`在scala api中仅仅是`Dataset[ROW]`类型的别名

In [1]:
// org.apache.spark.sql.Dataset
val df = spark.read.json("people.json")
println(df.getClass)
df.show()

class org.apache.spark.sql.Dataset
+----+-------+
| age|   name|
+----+-------+
|  30|   Andy|
|null|Michael|
|  19| Justin|
+----+-------+



df = [age: bigint, name: string]


[age: bigint, name: string]

#### 2. 无类型的Dataset操作 (DataFrame)
1. 因为Dataset是一组java对象组成的, 这些对象是强类型的, 其设计思路与RDD一致.  
2. DataFrame概念上被设计成无类型的Dataset,即多个"Row"对象组成的Dataset, 即DataFrame=Dataset[Row]

In [ ]:
// Print the schema in a tree format
df.printSchema()
// root
// |-- age: long (nullable = true)
// |-- name: string (nullable = true)

// Select only the "name" column
df.select("name").show()
// +-------+
// |   name|
// +-------+
// |Michael|
// |   Andy|
// | Justin|
// +-------+

// Select everybody, but increment the age by 1
df.select($"name", $"age" + 1).show()
// +-------+---------+
// |   name|(age + 1)|
// +-------+---------+
// |Michael|     null|
// |   Andy|       31|
// | Justin|       20|
// +-------+---------+

// Select people older than 21
df.filter($"age" > 21).show()
// +---+----+
// |age|name|
// +---+----+
// | 30|Andy|
// +---+----+

// Count people by age
df.groupBy("age").count().show()
// +----+-----+
// | age|count|
// +----+-----+
// |  19|    1|
// |null|    1|
// |  30|    1|
// +----+-----+

#### 2. Running SQL Queries on DataFrames

1. SparkSession 的 sql 函数可以让应用程序以编程的方式运行 SQL 查询, 并将结果作为一个 DataFrame 返回.

In [2]:
// Register the DataFrame as a SQL temporary view
df.createOrReplaceTempView("people")

val sqlDF = spark.sql("SELECT * FROM people")
sqlDF.show()

+----+-------+
| age|   name|
+----+-------+
|  30|   Andy|
|null|Michael|
|  19| Justin|
+----+-------+



sqlDF = [age: bigint, name: string]


[age: bigint, name: string]

2. 全局临时视图  
Spark SQL中的临时视图是session级别的, 也就是会随着session的消失而消失. 如果你想让一个临时视图在所有session中相互传递并且可用, 直到Spark 应用退出, 你可以建立一个全局的临时视图.全局的临时视图存在于系统数据库 global_temp中, 我们必须加上库名去引用它, 比如. SELECT * FROM global_temp.view1.

In [ ]:
// Register the DataFrame as a global temporary view
df.createGlobalTempView("people")

// Global temporary view is tied to a system preserved database `global_temp`
spark.sql("SELECT * FROM global_temp.people").show()
// +----+-------+
// | age|   name|
// +----+-------+
// |null|Michael|
// |  30|   Andy|
// |  19| Justin|
// +----+-------+

// Global temporary view is cross-session
spark.newSession().sql("SELECT * FROM global_temp.people").show()
// +----+-------+
// | age|   name|
// +----+-------+
// |null|Michael|
// |  30|   Andy|
// |  19| Justin|
// +----+-------+

### 2.2 Datasets
#### 1. Creating Datasets
1. Datasets和RDD相似, 但是Datasets没有使用java序列化或`Kryo`序列化. 它使用`Encoder`将对象序列化称bytes.  
 `encoders`可以动态产生代码, 在集群网络中传递, 而且Spark不需要反序列化就能在这些对象上执行filter,sort等操作  
2. Dataset可由`Seq`生成, 也可从DataFrame转化而来

In [ ]:
val myspark = spark
import myspark.implicits._

// Note: Case classes in Scala 2.10 can support only up to 22 fields. To work around this limit,
// you can use custom classes that implement the Product interface
// case class Person(name: Option[String], age: Option[Long])
case class Person(name: String, age: Long)

// Encoders are created for case classes
val caseClassDS = Seq(Person("Andy", 32)).toDS()
caseClassDS.show()
// +----+---+
// |name|age|
// +----+---+
// |Andy| 32|
// +----+---+


// Encoders for most common types are automatically provided by importing spark.implicits._
val primativeDS = Seq(1,2,3).toDS
primativeDS.map(x=>x+1).collect // Returns: Array(2, 3, 4)


// DataFrames can be converted to a Dataset by providing a class. Mapping will be done by name
val path = "people.json"
val peopleDS = spark.read.json(path).as[Person]
peopleDS.collect
// res4: Array[Person] = Array(Person(Some(Michael),None), 
//                             Person(Some(Andy),Some(30)), 
//                             Person(Some(Justin),Some(19))
//                            )


### 2.3 RDD互操作  
Spark SQL 支持两种不同的方法用于转换已存在的RDD成为Dataset :  
1. 第一种方法是使用反射去推断一个包含指定的对象类型的 RDD 的 Schema   
2. 第二种用于创建 Dataset 的方法是通过一个允许你构造一个 Schema 然后把它应用到一个已存在的 RDD 的编程接口.    

#### 1. 利用反射推断schama  
1. Spark SQL的Scala接口支持自动转换一个包含 case classes的RDD为DataFrame.  
2. Case class定义了表的Schema.Case class的参数名使用反射读取并且成为了列名.  
3. Case class也可以是嵌套的或者包含像Seq或者Array这样的复杂类型.这个RDD能够被隐式转换成一个DataFrame然后被注册为一个表.表可以用于后续的SQL语句.

In [20]:
val myspark = spark
import myspark.implicits._

case class Person(name: String, age: Long)

val rdd1 = spark.sparkContext.textFile("people.txt")
val rdd2 = rdd1.map(x=>x.split(",")).map(x=>Person(x(0),x(1).trim.toInt)) // Rdd[Person]  
println(rdd2.getClass)
rdd2.collect.foreach(print)
val peopleDF = rdd2.toDF
peopleDF.collect.foreach(println)

// [Michael,29]
// [Andy,30]
// [Justin,19]


// Register the DataFrame as a temporary view
peopleDF.createOrReplaceTempView("people")

// SQL statements can be run by using the sql methods provided by Spark
val teenagersDF = spark.sql("SELECT name, age FROM people WHERE age BETWEEN 13 AND 19")

// The columns of a row in the result can be accessed by field index
val rdd_1 = teenagersDF.map(row => "Name:"+row(0))
rdd_1.collect.foreach(println)
// Name:Justin
// Name:Justin

// or by field name
val rdd_2 = teenagersDF.map(row => "Name:"+row.getAs[String]("name"))
rdd_2.collect.foreach(println)


// No pre-defined encoders for Dataset[Map[K,V]], define explicitly
implicit val mapEncoder = org.apache.spark.sql.Encoders.kryo[Map[String,Any]]
val rdd_3 = teenagersDF.map(row=>row.getValuesMap[Any](List("name","age")))
rdd_3.collect.foreach(println)
//Map(name -> Justin, age -> 19)

class org.apache.spark.rdd.MapPartitionsRDD
Person(Michael,29)Person(Andy,30)Person(Justin,19)[Michael,29]
[Andy,30]
[Justin,19]
Name:Justin
Name:Justin
Map(name -> Justin, age -> 19)


myspark = org.apache.spark.sql.SparkSession@8488041
defined class Person
rdd1 = people.txt MapPartitionsRDD[103] at textFile at <console>:33
rdd2 = MapPartitionsRDD[105] at map at <console>:34
peopleDF = [name: string, age: bigint]
teenagersDF = [name: string, age: bigint]
rdd_1 = [value: string]
rdd_2 = [value: string]
mapEncoder = class[value[0]: binary]
rdd_3 = [value: binary]


[value: binary]

#### 2. 构造StructType对象
1. 从原始RDD中创建RDD[ROW]  
2. 创建StructType匹配RDD中的Row  
3. 通过 SparkSession 提供的 createDataFrame 方法应用 Schema 到 RDD 的 RowS（行）

In [1]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.Row
val path = "/Users/lj/devkits/spark-2.3.0-bin-hadoop2.7/examples/src/main/resources/people.txt"
// StructType
val schema = StructType(Array(StructField("name",StringType,nullable=true),
                             StructField("age",StringType,nullable=true)))
// Origion RDD
val rdd1 = spark.sparkContext.textFile(path)
val rdd2 = rdd1.map(x=>x.split(","))
val rdd3 = rdd2.map(x=>Row(x(0),x(1).trim))  // RDD[Row]

//DataFrame
val peopleDF = spark.createDataFrame(rdd3,schema)  // RDD to Dataframe
peopleDF.createOrReplaceTempView("people")

val res = spark.sql("select collect_list(age) from people")
res.collect

Array([WrappedArray(29, 30, 19)])

#### 3. pivot
1. 构造数据
2. 透视的基本写法  
```
df
  .groupBy(grouping_columns)
  .pivot(pivot_column, [values]) 
  .agg(aggregate_expressions)
  ```

In [31]:
case class Item(date:String,program:String,income:Int)
val myspark = spark
import myspark.implicits._

val rdd1 = sc.parallelize(List(Item("2018-01","项目1",100),
                              Item("2018-01","项目2",200),
                              Item("2018-01","项目3",300),
                              Item("2018-02","项目1",1000),
                              Item("2018-03","项目x",999),
                              Item("2018-02","项目2",2000)))
val df = rdd1.toDF
df.collect

defined class Item
myspark = org.apache.spark.sql.SparkSession@8488041
rdd1 = ParallelCollectionRDD[129] at parallelize at <console>:29
df = [date: string, program: string ... 1 more field]


lastException: Throwable = null


2018-01,项目1,100
2018-01,项目2,200
2018-01,项目3,300
2018-02,项目1,1000
2018-03,项目x,999
2018-02,项目2,2000


In [40]:
import org.apache.spark.sql.functions.avg
df.groupBy("date")
  .pivot("program")
  .agg(avg($"income"))
  .collect

2018-03,NULL,NULL,NULL,999.0
2018-02,1000.0,2000.0,NULL,NULL
2018-01,100.0,200.0,300.0,NULL


#### 4. 时间窗口函数
df1为用户展示表, df2为用户点击表. 统计每次展示给用户3天内, 用户的点击次数

In [1]:
val myspark = spark
import myspark.implicits._

In [2]:
// 推送表
val df1 = Seq(("user1","item1","2018-5-6"),
             ("user1","item2","2018-5-7"),
             ("user2","item1","2018-5-4"),
             ("user2","item2","2018-5-6")).toDF("userId","itemId","time")

// 点击表
val df2 = Seq(("user1","item1","2018-5-7"),
             ("user1","item1","2018-5-10"),
             ("user1","item2","2018-5-9"),
             ("user1","item2","2018-5-7"),
             ("user2","item1","2018-5-5"),
             ("user2","item2","2018-5-7")).toDF("userId","itemId","time")
df1.show
df2.show

+------+------+--------+
|userId|itemId|    time|
+------+------+--------+
| user1| item1|2018-5-6|
| user1| item2|2018-5-7|
| user2| item1|2018-5-4|
| user2| item2|2018-5-6|
+------+------+--------+

+------+------+---------+
|userId|itemId|     time|
+------+------+---------+
| user1| item1| 2018-5-7|
| user1| item1|2018-5-10|
| user1| item2| 2018-5-9|
| user1| item2| 2018-5-7|
| user2| item1| 2018-5-5|
| user2| item2| 2018-5-7|
+------+------+---------+



In [3]:
import org.apache.spark.sql.functions._
// 先用to_date将时间转换成日期类型
val df1_cast = df1.select($"userId",$"itemId",to_date($"time","yyyy-MM-dd") as "time_df1")
val df2_cast = df2.select($"userId",$"itemId",to_date($"time","yyyy-MM-dd") as "time_df2")

df1_cast.createOrReplaceTempView("t1")
df2_cast.createOrReplaceTempView("t2")

In [27]:
spark.sql("SELECT t.userId,t.itemId,count(1) FROM (
              SELECT t1.userId,t1.itemId,t1.time_df1,t2.time_df2,datediff(time_df2,time_df1) as delay 
                  from t1 left join t2 
                  on t1.userId=t2.userId and t1.itemId=t2.itemId 
                  where datediff(time_df2,time_df1) between 0 and 3
           )as t group by t.userId,t.itemId,t.time_df1").show

+------+------+--------+
|userId|itemId|count(1)|
+------+------+--------+
| user2| item2|       1|
| user1| item2|       2|
| user2| item1|       1|
| user1| item1|       1|
+------+------+--------+



In [34]:
// 上个例子用date_diff计算日期差距, 这里用unix_timestamp计算日期差距
spark.sql("select *,unix_timestamp(time_df1)-unix_timestamp(time_df2) as diff 
           from t1 left join t2 
           on t1.userId=t2.userId and t1.itemId=t2.itemId")
.show

+------+------+----------+------+------+----------+-------+
|userId|itemId|  time_df1|userId|itemId|  time_df2|   diff|
+------+------+----------+------+------+----------+-------+
| user1| item1|2018-05-06| user1| item1|2018-05-10|-345600|
| user1| item1|2018-05-06| user1| item1|2018-05-07| -86400|
| user1| item2|2018-05-07| user1| item2|2018-05-07|      0|
| user1| item2|2018-05-07| user1| item2|2018-05-09|-172800|
| user2| item1|2018-05-04| user2| item1|2018-05-05| -86400|
| user2| item2|2018-05-06| user2| item2|2018-05-07| -86400|
+------+------+----------+------+------+----------+-------+



#### 5. 时间窗口函数
在我们介绍如何使用time window之前，我们先来准备一份时间序列数据。本文将使用Apple公司从1980年到2016年期间的股票交易信息。股票数据一共有六列，但是这里我们仅关心Date和Close两列，它们分别代表股票交易时间和当天收盘的价格。



In [34]:
import org.apache.spark.sql.functions._
val df = spark.read.
                option("header",true).
                option("inferSchema","true").
                csv("data/iteblog_apple.csv")
df.show

+-------------------+---------+---------+---------+---------+--------+---------+
|               Date|     Open|     High|      Low|    Close|  Volume|Adj Close|
+-------------------+---------+---------+---------+---------+--------+---------+
|2016-07-11 00:00:00|    96.75|97.650002|96.730003|96.980003|23298900|96.980003|
|2016-07-08 00:00:00|96.489998|96.889999|96.050003|    96.68|28855800|    96.68|
|2016-07-07 00:00:00|95.699997|     96.5|95.620003|95.940002|24280900|95.940002|
|2016-07-06 00:00:00|94.599998|95.660004|94.370003|95.529999|30770700|95.529999|
|2016-07-05 00:00:00|95.389999|95.400002|94.459999|95.040001|27257000|95.040001|
|2016-07-01 00:00:00|95.489998|96.470001|95.330002|95.889999|25872300|95.889999|
|2016-06-30 00:00:00|94.440002|95.769997|94.300003|95.599998|35836400|95.599998|
|2016-06-29 00:00:00|93.970001|94.550003|93.629997|94.400002|36531000|94.400002|
|2016-06-28 00:00:00|92.900002|93.660004|92.139999|93.589996|40444900|93.589996|
|2016-06-27 00:00:00|     93

#### 步骤一：找出2016年的股票交易数据 

In [35]:
val df_2016 = df.filter("year(Date)=2016")  //(year是dataframe里的function)

#### 步骤二：计算平均值
现在我们需要对每个星期创建一个窗口，这种类型的窗口通常被称为tumbling window，window一般在group by语句中使用。
window方法的第一个参数指定了时间所在的列；第二个参数指定了窗口的持续时间(duration)，它的单位可以是seconds、minutes、hours、days或者weeks。创建好窗口之后，我们可以计算平均值。

In [36]:
import org.apache.spark.sql.DataFrame
val myspark = spark
import myspark.implicits._   // 自动将$"col"转换为DataFrame的Column

In [37]:
val thumbling_df = df_2016.
                        groupBy(window($"Date","1 week")).
                        agg(avg($"Close").as("close_agg"))
                        
//print_window(thumbling_df,"Close")
thumbling_df.sort("window.start").select("window.start","window.end","close_agg").show

+-------------------+-------------------+------------------+
|              start|                end|         close_agg|
+-------------------+-------------------+------------------+
|2015-12-31 08:00:00|2016-01-07 08:00:00|101.30249774999999|
|2016-01-07 08:00:00|2016-01-14 08:00:00| 98.47199859999999|
|2016-01-14 08:00:00|2016-01-21 08:00:00| 96.72000125000001|
|2016-01-21 08:00:00|2016-01-28 08:00:00|        97.6719984|
|2016-01-28 08:00:00|2016-02-04 08:00:00|         96.239999|
|2016-02-04 08:00:00|2016-02-11 08:00:00| 94.39799819999999|
|2016-02-11 08:00:00|2016-02-18 08:00:00|        96.2525005|
|2016-02-18 08:00:00|2016-02-25 08:00:00| 96.09400000000001|
|2016-02-25 08:00:00|2016-03-03 08:00:00|         99.276001|
|2016-03-03 08:00:00|2016-03-10 08:00:00|101.64000100000001|
|2016-03-10 08:00:00|2016-03-17 08:00:00|        104.226001|
|2016-03-17 08:00:00|2016-03-24 08:00:00|       106.0699996|
|2016-03-24 08:00:00|2016-03-31 08:00:00|       107.8549995|
|2016-03-31 08:00:00|201

#### 带有开始时间的Time window
上面的输出按照window.start进行了排序，这个字段标记了窗口的开始时间。上面的输出你可能已经看到了第一行的开始时间是2015-12-31，结束时间是2016-01-07。但是你从原始数据可以得到：2016年Apple公司的股票交易信息是从2016-01-04开始的；原因是2016-01-01是元旦，而2016-01-02和2016-01-03正好是周末，期间没有股票交易。

我们可以手动指定窗口的开始时间来解决这个问题。
```scala
def window(timeColumn: Column, windowDuration: String, slideDuration: String, startTime: String){
    Bucketize rows into one or more time windows given a timestamp specifying column. Window starts are inclusive but the window ends are exclusive, e.g. 12:05 will be in the window [12:05,12:10) but not in [12:00,12:05). Windows can support microsecond precision. Windows in the order of months are not supported. The following example takes the average stock price for a one minute window every 10 seconds starting 5 seconds after the hour:
}
```


In [8]:
// 下面的示例中，4 days参数就是开始时间的偏移量；前两个参数分别代表窗口时间和滑动时间，我们打印出这个窗口的内容：
val iteblogWindowWithStartTime = df_2016.
                                    groupBy(window($"Date","1 week","1 week","4 days")).
                                    agg(avg($"Close").as("close_agg"))
iteblogWindowWithStartTime.sort("window.start").select("window.start","window.end","close_agg").show

+-------------------+-------------------+------------------+
|              start|                end|         close_agg|
+-------------------+-------------------+------------------+
|2015-12-28 08:00:00|2016-01-04 08:00:00|        105.349998|
|2016-01-04 08:00:00|2016-01-11 08:00:00|        99.0699982|
|2016-01-11 08:00:00|2016-01-18 08:00:00| 98.49999799999999|
|2016-01-18 08:00:00|2016-01-25 08:00:00|        98.1220016|
|2016-01-25 08:00:00|2016-02-01 08:00:00|        96.2539976|
|2016-02-01 08:00:00|2016-02-08 08:00:00| 95.29199960000001|
|2016-02-08 08:00:00|2016-02-15 08:00:00|        94.2374975|
|2016-02-15 08:00:00|2016-02-22 08:00:00|        96.7880004|
|2016-02-22 08:00:00|2016-02-29 08:00:00| 96.23000160000001|
|2016-02-29 08:00:00|2016-03-07 08:00:00|101.53200079999999|
|2016-03-07 08:00:00|2016-03-14 08:00:00|       101.6199998|
|2016-03-14 08:00:00|2016-03-21 08:00:00|105.63600160000001|
|2016-03-21 08:00:00|2016-03-28 08:00:00|105.92749950000001|
|2016-03-28 08:00:00|201

#### 6. Window api
有如下表, We want to answer two questions:
1. What are the best-selling and the second best-selling products in every category?  
2. What is the difference between the revenue of each product and the revenue of the best-selling product in the same category of that product?


In [2]:
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions._
val myspark = spark
import myspark.implicits._

val df = Seq(("Thin", "Cell Phone",6000), ("Normal", "Tablet",1500), ("Mini","Tablet",5500), 
             ("Ultra Thin","Cell Phone",5000),("Very Thin","Cell Phone",6000),("Big","Tablet",2500), 
             ("Bendable","Cell Phone",3000),("Foldable","Cell Phone",3000), 
             ("Pro","Tablet",4500),("Pro2","Tablet",6500)).
         toDF("product", "category","revenue")
df.show

+----------+----------+-------+
|   product|  category|revenue|
+----------+----------+-------+
|      Thin|Cell Phone|   6000|
|    Normal|    Tablet|   1500|
|      Mini|    Tablet|   5500|
|Ultra Thin|Cell Phone|   5000|
| Very Thin|Cell Phone|   6000|
|       Big|    Tablet|   2500|
|  Bendable|Cell Phone|   3000|
|  Foldable|Cell Phone|   3000|
|       Pro|    Tablet|   4500|
|      Pro2|    Tablet|   6500|
+----------+----------+-------+



#### 第一个问题: 每个category内的前两名

In [4]:
// dense_rank():window内的rank按照实际大小递增, 相同的值有相同的rank
// rank(): window内的rank按照序列顺序递增, 相同的值会有不同的rank
df.createOrReplaceTempView("revenue_df")
spark.sql("SELECT * FROM (
                SELECT product,
                       category,
                       revenue, 
                       dense_rank() OVER (PARTITION BY category ORDER BY revenue DESC) as rank 
                FROM revenue_df 
        )WHERE rank<=2").show

+----------+----------+-------+----+
|   product|  category|revenue|rank|
+----------+----------+-------+----+
|      Pro2|    Tablet|   6500|   1|
|      Mini|    Tablet|   5500|   2|
|      Thin|Cell Phone|   6000|   1|
| Very Thin|Cell Phone|   6000|   1|
|Ultra Thin|Cell Phone|   5000|   2|
+----------+----------+-------+----+



#### 第二个问题: 每个category内与自己category内最大的值得差距

In [4]:
// DataFrame API
val window_space = Window.partitionBy($"category").orderBy($"revenue".desc)
val dif = max($"revenue").over(window_space)-$"revenue"
df.select($"product",$"category",$"revenue",dif.as("dif")).show

+----------+----------+-------+----+
|   product|  category|revenue| dif|
+----------+----------+-------+----+
|      Pro2|    Tablet|   6500|   0|
|      Mini|    Tablet|   5500|1000|
|       Pro|    Tablet|   4500|2000|
|       Big|    Tablet|   2500|4000|
|    Normal|    Tablet|   1500|5000|
|      Thin|Cell Phone|   6000|   0|
| Very Thin|Cell Phone|   6000|   0|
|Ultra Thin|Cell Phone|   5000|1000|
|  Bendable|Cell Phone|   3000|3000|
|  Foldable|Cell Phone|   3000|3000|
+----------+----------+-------+----+



In [6]:
// sql
spark.sql("SELECT * FROM (
            SELECT product,
                   category,
                   revenue, 
                   max(revenue) OVER (PARTITION BY category ORDER BY revenue DESC)-revenue as dif 
            FROM revenue_df )").show

+----------+----------+-------+----+
|   product|  category|revenue| dif|
+----------+----------+-------+----+
|      Pro2|    Tablet|   6500|   0|
|      Mini|    Tablet|   5500|1000|
|       Pro|    Tablet|   4500|2000|
|       Big|    Tablet|   2500|4000|
|    Normal|    Tablet|   1500|5000|
|      Thin|Cell Phone|   6000|   0|
| Very Thin|Cell Phone|   6000|   0|
|Ultra Thin|Cell Phone|   5000|1000|
|  Bendable|Cell Phone|   3000|3000|
|  Foldable|Cell Phone|   3000|3000|
+----------+----------+-------+----+

